In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import matplotlib.patches
import numpy as np
from pyprind import prog_percent
import random
from skimage.io import imread, imsave
from zebrafishframework import io
from zebrafishframework import rendering
from zebrafishframework import segmentation

from importlib import reload

In [ ]:
rendering = reload(rendering)
segmentation = reload(segmentation)

In [ ]:
#base = '/Users/koesterlab/Registrations/series/calcium/fish18_6dpf_medium'
base = '/Users/koesterlab/registered/control/fish22_6dpf_medium'
roi_file = base + '_rois.pickle'
rendering_file = base + '_rendering.mp4'
template = imread(r"/Users/koesterlab/andreas_code/cell.tif")
anatomy      = imread(base + "_std_div.tif")
#mask         = imread(base + "_anatomy_mask.tif") > 0
mask = np.ones(anatomy.shape) # no mask
#displacement = np.load(base + "_registration_meta.npy")
#trace        = dd.io.load(base + "_aligned.h5")['stack']
trace        = io.read_h5py(base + '.hdf5')

In [ ]:
anatomy_std = segmentation.std(trace, anatomy)

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(anatomy_std[10],aspect='auto')

In [ ]:
rois = segmentation.find_rois_andreas(anatomy_std, template=template, trace=trace)

In [ ]:
roi_map = segmentation.draw_rois(rois, anatomy_std)
@interact
def browse(i:(0,20)):
    plt.figure(figsize=(12,12))
    plt.imshow(roi_map[i])

In [ ]:
import pickle
with open(roi_file, 'wb') as f:
    pickle.dump(rois, f)

In [ ]:
traces_dFF = segmentation.dFF(np.array([roi['trace'] for roi in rois]), np.arange(100, 160))

In [ ]:
traces_dFF_backup = traces_dFF.copy()

Sanity checking. Render rois over source

In [ ]:
ts = np.arange(800, 1000, 1)
zs = np.array([10])
activity = np.zeros(tuple(ts.shape) + tuple(zs.shape) + tuple(trace.shape[2:]) + (3,), dtype=trace.dtype)

In [ ]:
for i, t in prog_percent(list(enumerate(ts))):
    activity[i,:,:,:] = np.expand_dims(trace[t][zs], axis=3)

In [ ]:
range_dFF = np.max(traces_dFF)
dFF_factor = 1000
dFF_off = 2

In [ ]:
from skimage.draw import circle
radius = 2

# zmap[z] is index of plane z in zs
zmap = [np.where(zs == i)[0][0] if i in zs else -1 for i in range(trace.shape[1])]

filtered_rois = list(filter(lambda roi: roi['z'] in zs, rois))

for i, t in prog_percent(list(enumerate(ts))):
    for roi_id, v in list(enumerate(filtered_rois)):
        x, y = v['x'], v['y']
        z = zmap[v['z']]
        #circle_xy = circle(x, y, radius, activity.shape[2:4])
        c = (traces_dFF[roi_id, t] + dFF_off)*dFF_factor
        #        c = colors[np.argmin((vals-traces_dFF[roi_id, t])**2)]
        activity[i][z][x][y][0] = 2000


In [ ]:
!rm sanity_check_tzyxc.h5

In [ ]:
import h5py
h5f = h5py.File('sanity_check_tzyxc.h5')
h5f.create_dataset('stack', data=activity)
h5f.flush()
h5f.close()

Unsorted dFF

In [ ]:
plt.figure(figsize=(12,10))
plt.imshow(traces_dFF,aspect='auto', vmax=2)
plt.colorbar()

In [ ]:
plt.figure(figsize=(12,10))
plt.imshow(traces_dFF[np.argsort(np.average(traces_dFF, axis=1))],aspect='auto', vmax=2)
#plt.xlim((600, 800))
plt.colorbar()

In [ ]:
F = np.array([roi['trace'] for roi in rois])
mean_F = np.mean(F, axis=0)
mean_trace = np.mean(filtered_dFF, axis=0)

import scipy.stats

(a_s, b_s, r, tt, stderr) = scipy.stats.linregress(ts[:], np.log(mean_F[:]))
reg = np.exp(b_s+a_s*ts)

plt.figure(figsize=(12,10))
#plt.ylim((0, 350))
#plt.plot(ts, mean_F, 'blue')
#plt.plot(ts, reg, 'red')
#plt.plot(ts, mean_trace-.1*std_trace, 'red', alpha=.5)
#plt.plot(ts, mean_trace+.1*std_trace, 'red', alpha=.5)

# from some controls
a_s = -0.00015

factors = np.exp(-a_s*ts)

F_adjusted = F*factors
dFF_adjusted = segmentation.dFF(F_adjusted, np.arange(100, 160))

plt.plot(ts, np.mean(F, axis=0))

In [ ]:
a_s

In [ ]:
plt.figure(figsize=(12,10))
#plt.ylim((-1, 2))
plt.xlim((800, 1000))

ts = np.arange(F.shape[1])
choices = np.arange(np.alen(F))
random.shuffle(choices)
choices = choices[:100]

chosen = F[choices]

for trc in chosen:
    plt.plot(ts, trc, color='red', alpha=.05)
"""

for filt in filtered:
    plt.plot(ts, filt, color='black', alpha=.05)

"""

Render traces

In [ ]:
rendering = reload(rendering)
render_ts = np.arange(0, 1800, 1)

def color_func(dFF):
    final_a = (0, 255, 0)
    final_b = (255, 0, 255)
    alpha = 1
    max_dFF = 1
    c = np.array(final_b if dFF > 0 else final_a, dtype=np.float32)
    dFF = min(abs(dFF), max_dFF)/max_dFF
    return np.array(c*alpha*dFF, dtype=np.uint8)

activity = rendering.orthogonal(rois, filtered_dFF, color_func, render_ts, (1024, 1024))
plt.figure(figsize=(12,10))
plt.imshow(np.array(activity[0], dtype=np.uint8))


In [ ]:
rendering.to_file(base + '_rendering.mp4', activity)

In [ ]:
#plt.figure(figsize=(12,10))
def visualize_clustering(dFF, clustering):
    n = np.max(clustering) + 1
    cmap = plt.cm.get_cmap('hsv', n+1)
    color_func = lambda roi_id: np.array(cmap(km.labels_[roi_id])[:3])*255
    
    for cluster in range(n):
        fig = plt.figure(figsize=(12,5))
        cluster_traces = dFF[clustering == cluster]
        img = plt.imshow(cluster_traces[np.argsort(np.average(cluster_traces, axis=1))],aspect='auto', vmax=2)
        plt.title('cluster %d' % (cluster + 1), color=cmap(cluster))
        plt.colorbar()
    
    t = np.arange(traces_dFF.shape[1])
    """
    bins = np.arange(2, -1, .01)
    im = np.zeros((bins.shape[0], t.shape[0], 3))
    im.axes.yaxis.
    """

    plt.figure(figsize=(12,10))
    
    plt.ylim((-1, 2))
    for cluster in range(n):
        cluster_traces = dFF[clustering == cluster]
        random.shuffle(cluster_traces)
        for trace in cluster_traces[:200]:
            plt.plot(t, trace, color=cmap(cluster), alpha=.005)
    
    roi_map = segmentation.draw_rois(rois, anatomy_std, color_func=color_func)
    @interact
    def browse(i:(0,20)):
        plt.figure(figsize=(12,12))
        plt.imshow(roi_map[i])


Sorted dFF

In [ ]:
indices = np.argsort(np.average(filtered_dFF[:, 1600:], axis=1))

def map_to_group(index, n):
    percentile = index/n
    if percentile < .2:
        return 0
    if percentile > .8:
        return 2
    return 1

clustering = np.array(list(map(lambda e: map_to_group(e, filtered_dFF.shape[0]), np.argsort(indices))))

"""
plt.figure(figsize=(12,10))
plt.imshow(traces_dFF[indices],aspect='auto', vmax=2)
plt.colorbar()
"""



In [ ]:
%%time
visualize_clustering(filtered_dFF, clustering)

Clustered traces

In [ ]:
%%time
n = 10
from sklearn.cluster import KMeans
km = KMeans(n)
km.fit(filtered_dFF[:,1000:1600])

In [ ]:
%%time
visualize_clustering(filtered_dFF, km.labels_)

In [ ]:
roi_map = segmentation.draw_rois(rois, anatomy_std, color_func=color_func)
@interact
def browse(i:(0,20)):
    plt.figure(figsize=(12,12))
    plt.imshow(roi_map[i])

In [ ]:
mean_before = traces_dFF[:,100:160].mean(1)
mean_after  = traces_dFF[:,700:].mean(1)
bins = np.arange(-1, 2, .02)
before_hist = np.histogram(mean_before, bins=bins)
after_hist = np.histogram(mean_after, bins=bins)

In [ ]:
plt.figure(figsize=(12, 10))
#plt.hist(mean_before, bins, alpha=.5, label='before')
plt.hist(mean_after, bins, alpha=.5, label='after')
plt.legend(loc=0)
plt.show()